# **Training HRPlane Dataset with YOLOv9e: Experiment No. 62.2**

<p align="center">
    <img src="https://github.com/RSandAI/Comprehensive-YOLO-Airplane-Detection/blob/main/assets/image.png" height=450 width=1280 alt=""/>
</p>

<small>Picture Source: <a href="https://github.com/RSandAI/Comprehensive-YOLO-Airplane-Detection/">RSandAI, Comprehensive YOLO Airplane Detection</a></small>

<br>

## **Context**

The HRPlanes dataset is a valuable resource in the domain of computer vision, particularly for tasks related to object detection, specifically focusing on aircraft within Very High Resolution (VHR) Google Earth images. This dataset comprises 3101 RGB images of major airports and aircraft boneyards, manually annotated with bounding boxes for airplanes using Plainsight (formerly HyperLabel). Quality control was conducted independently, resulting in 18,477 annotated airplanes. The dataset is split into 70% training (2170 images), 20% validation (620 images), and 10% testing (311 images) sets.

<br>

To facilitate accurate training and evaluation, each aircraft within the images has been meticulously labeled, resulting in a comprehensive annotation set covering a total of 18,477 aircraft instances. These annotations provide bounding box coordinates for each detected aircraft within the corresponding images.

<br>

Given the specifics of the HRPlanes dataset and its annotations, training object detection models like YOLOv9 would involve leveraging this rich dataset to teach the model to accurately detect and localize aircraft within VHR Google Earth images. This process would entail feeding the dataset into the YOLOv9 model architecture and iteratively training the model until satisfactory performance metrics are achieved. Access to the HRPlanes dataset provides researchers, practitioners, and developers with a valuable asset for advancing the state-of-the-art in aircraft detection algorithms, with potential applications spanning aviation security, urban planning, and defense intelligence. The dataset's availability ensures its accessibility to the wider computer vision community, fostering collaboration and innovation in this domain.

[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/WongKinYiu/yolov9)
[![arXiv](https://img.shields.io/badge/arXiv-2402.13616-b31b1b.svg)](https://arxiv.org/pdf/2402.13616.pdf)

**Hyperparameters:**

| Epochs | Network size | Batch size | LR | Optimizer | Hue | Saturation | Value | Mosiac |
|--|--|--|--|--|--|--|--|--|
| 100 | 640x640 | 16 | 0.001 | AdamW | 0.015 | 0.7 | 0.4 | 1.0 |

<br>

Make sure your runtime is **GPU** (_not_ CPU or TPU). And if it is an option, make sure you are using _Python 3_. You can select these settings by going to `Runtime -> Change runtime type -> Select the above mentioned settings and then press SAVE`.

## **0. Initial Steps**

### **0.1. Import Libraries and Connect Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
import json
import shutil
import seaborn as sns
import os

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [ ]:
HOME = os.getcwd()
print(HOME)

/content


### **0.2. Define Paths and Hyperparameters**

In [ ]:
# @markdown ---

BASE_DIRECTORY = '/gdrive/MyDrive/Datasets/HRPlanes/YOLOV8/YOLOv9e/Experiment_No_62/' # @param {type:"string"}
if os.path.exists(BASE_DIRECTORY):
    print(f"Base directory already exists: {BASE_DIRECTORY}")
else:
    os.makedirs(BASE_DIRECTORY)
    print(f"Created base directory: {BASE_DIRECTORY}")

SAVE_DIR = BASE_DIRECTORY + 'outputs'
if os.path.exists(SAVE_DIR):
    print(f"Save directory already exists: {SAVE_DIR}")
else:
    os.makedirs(SAVE_DIR)
    print(f"Created save directory: {SAVE_DIR}")

# @markdown ---

IMAGE_SIZE = 640 # @param {type:"integer"}
BATCH = 16 # @param {type:"integer"}
# SGD, Adam, AdamW
OPTIMIZER = 'AdamW' # @param {type:"string"}

# @markdown ---

Base directory already exists: /gdrive/MyDrive/Datasets/HRPlanes/YOLOV8/YOLOv9e/Experiment_No_62/
Created save directory: /gdrive/MyDrive/Datasets/HRPlanes/YOLOV8/YOLOv9e/Experiment_No_62/outputs


### **0.3. Clone and Install**

**NOTE:** YOLOv9 is very new. At the moment, we recommend using a fork of the main repository. The `detect.py` script contains a bug that prevents inference. This bug is patched in the fork.

In [ ]:
!git clone https://github.com/WongKinYiu/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q

Cloning into 'yolov9'...
remote: Enumerating objects: 668, done.
remote: Counting objects: 100% (294/294), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 668 (delta 224), reused 201 (delta 201), pack-reused 374
Receiving objects: 100% (668/668), 3.22 MiB | 7.14 MiB/s, done.
Resolving deltas: 100% (269/269), done.
/content/yolov9
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 11.5 MB/s eta 0:00:00


### **0.4. Download Model Weights**

**NOTE:** In the YOLOv9 paper, versions `yolov9-s` and `yolov9-m` are also mentioned, but the weights for these models are not yet available in the YOLOv9 [repository](https://github.com/WongKinYiu/yolov9).

In [ ]:
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt

In [ ]:
!ls -la {HOME}/weights

total 136940
drwxr-xr-x 2 root root      4096 May 31 11:31 .
drwxr-xr-x 1 root root      4096 May 31 11:31 ..
-rw-r--r-- 1 root root 140217688 Feb 18 12:36 yolov9-e.pt


**NOTE:** The dataset must be saved inside the `{HOME}/yolov9` directory, otherwise, the training will not succeed.

## **1. Train the Model**

The A100 GPU is a powerful graphics processing unit (GPU) developed by NVIDIA. It is part of the NVIDIA Ampere architecture and is designed for high-performance computing tasks, including deep learning, data analytics, and scientific computing. The A100 GPU offers significant improvements in performance and efficiency compared to previous GPU models, making it ideal for demanding AI and machine learning applications.

In [ ]:
!nvidia-smi

Fri May 31 11:31:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

**IMPORTANT:** Before training, check hyperparameters in `/content/yolov9/data/hyps/hyp.scratch-high.yaml`.

In [ ]:
# !rm -rf /content/yolov9/runs/train/exp

In [ ]:
WEIGHT = "/gdrive/MyDrive/Datasets/HRPlanes/YOLOV8/YOLOv9e/Experiment_No_62/epoch_0_25/runs/train/exp/weights/best.pt"

In [ ]:
%cd {HOME}/yolov9

!python train_dual.py \
--batch-size {BATCH} --epochs 25 --imgsz {IMAGE_SIZE} --device 0 \
--data /gdrive/MyDrive/Datasets/HRPlanes/YOLOV8/coco.yaml \
--weights {WEIGHT} \
--cfg models/detect/yolov9-e.yaml \
--hyp hyp.scratch-high.yaml \
--optimizer {OPTIMIZER}

/content/yolov9
2024-05-31 11:31:51.863090: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-31 11:31:51.916854: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-31 11:31:51.916909: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-31 11:31:51.919014: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-31 11:31:51.927814: I tensorflow/core/platform/

## **2. Examine Training Results**

**NOTE:** By default, the results of each subsequent training sessions are saved in `{HOME}/yolov9/runs/train/`, in directories named `exp`, `exp2`, `exp3`, ... You can override this behavior by using the `--name` parameter.

In [ ]:
!ls {HOME}/yolov9/runs/train/exp/

confusion_matrix.png				    PR_curve.png	   val_batch0_pred.jpg
events.out.tfevents.1717155114.76afbcdbb188.3977.0  R_curve.png		   val_batch1_labels.jpg
F1_curve.png					    results.csv		   val_batch1_pred.jpg
hyp.yaml					    results.png		   val_batch2_labels.jpg
labels_correlogram.jpg				    train_batch0.jpg	   val_batch2_pred.jpg
labels.jpg					    train_batch1.jpg	   weights
opt.yaml					    train_batch2.jpg
P_curve.png					    val_batch0_labels.jpg


In [ ]:
csv_file_path = f"{HOME}/yolov9/runs/train/exp/results.csv"
df = pd.read_csv(csv_file_path)
df

,epoch,train/box_loss,train/cls_loss,train/dfl_loss,metrics/precision,metrics/recall,metrics/mAP_0.5,metrics/mAP_0.5:0.95,val/box_loss,val/cls_loss,val/dfl_loss,x/lr0,x/lr1,x/lr2
0,0,0.94316,0.47593,1.1554,0.98398,0.97067,0.99122,0.85201,0,0,0,0.067243,0.000331,0.000331
1,1,0.96471,0.48983,1.1642,0.98225,0.97386,0.99079,0.82856,0,0,0,0.034216,0.000638,0.000638
2,2,1.00810,0.51141,1.1823,0.97559,0.97172,0.99086,0.83463,0,0,0,0.001164,0.000919,0.000919
3,3,1.01530,0.51760,1.1807,0.98401,0.97325,0.99023,0.83835,0,0,0,0.000881,0.000881,0.000881
4,4,0.99920,0.50443,1.1821,0.98140,0.97848,0.99152,0.83560,0,0,0,0.000881,0.000881,0.000881
5,5,0.98369,0.50214,1.1798,0.98410,0.97812,0.99241,0.83929,0,0,0,0.000842,0.000842,0.000842
6,6,0.97466,0.49542,1.1755,0.98393,0.97872,0.99165,0.84762,0,0,0,0.000802,0.000802,0.000802
7,7,0.98489,0.49297,1.1742,0.98482,0.98359,0.99273,0.83850,0,0,0,0.000762,0.000762,0.000762
8,8,0.95556,0.47717,1.1641,0.98522,0.97508,0.99128,0.84496,0,0,0,0.000723,0.000723,0.000723
9,9,0.93682,0.46462,1.1590,0.98678,0.97541,0.99263,0.85178,0,0,0,0.000683,0.000683,0.000683


## **3. Evaluate Model**

In [ ]:
# %cd {HOME}/yolov9

# !python val_dual.py \
# --img 640 --batch 16 \
# --data /gdrive/MyDrive/Datasets/HRPlanes/YOLOV8/coco.yaml \
# --weights {HOME}/yolov9/runs/train/exp/weights/best.pt

## **4. Move Model Items into Google Drive**

In [ ]:
BASE_DIRECTORY

'/gdrive/MyDrive/Datasets/HRPlanes/YOLOV8/YOLOv9e/Experiment_No_62/'

Instead of uploading and downloading files, we can directly move them to the desired path.

In [ ]:
shutil.move(f"{HOME}/yolov9/runs/", BASE_DIRECTORY)

'/gdrive/MyDrive/Datasets/HRPlanes/YOLOV8/YOLOv9e/Experiment_No_62/runs'

In [ ]:
from datetime import datetime
print(f"Changes have been made to the project on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Changes have been made to the project on 2024-05-31 14:17:13
